# Prescribed Burns data for SA and NSW
This notebook uses data that is publically available for download from data.gov.au. The data you need is in the Shapefiles folder of this repo, so you should be able to run this code right away. The original notebook for the burns project uses data from VIC as well but you won't be able to download that from data.gov.au because it's like a really big file and you have to ask for it. The aim is to learn how to manipulate spatial data with geopandas. 

In [2]:
#run imports for the system and geopandas. I've also imported a mapping module incase you want to map anything. 
import sys
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import ipyleaflet as ipyl
import ipywidgets as ipyw
import json


from collections import OrderedDict


cmaps = OrderedDict()

### Read the files and filter for prescribed burns
This will give you a dataset of prescribed burns for each state. Use the geopandas read_file() function.

In [4]:
#Read the shape files with geopandas
SA_gdf = gpd.read_file('Shapefiles/FIREMGT_LastFire_GDA94.shp')
NSW_gdf = gpd.read_file('Shapefiles/NPWS_FireHistory_20200326.shp')

### Alright let's have a look at the columns we have in these shapefiles

In [9]:
SA_gdf.head(1)

,INCIDENTNU,INCIDENTNA,INCIDENTTY,FIREDATE,FINANCIALY,FIREYEAR,SEASON,DATERELIAB,SHAPE_Leng,SHAPE_Area,geometry
0,0,None,Bushfire,1931-12-31,1931/1932,1931,SUMMER,6,0.199605,0.00021,"MULTIPOLYGON (((136.91089 -36.02206, 136.91090..."


In [10]:
NSW_gdf.head(1)

,FireName,FireNo,Label,StartDate,EndDate,AreaHa,PerimeterM,Verdate,geometry
0,None,None,1968-69 Prescribed Burn,1968-10-01,None,2.18477,1548.960666,26/03/2020,"POLYGON ((153.14143 -30.17730, 153.14144 -30.1..."


We have to filter the columns that say what kind of burn it was, whether it was a bushfire or a prescribed burn because we only want prescribed burns. We can see what unique values are in those columns. Then we can filter for only prescribed burns.

In [11]:
SA_gdf.INCIDENTTY.unique()

array(['Bushfire', 'Prescribed Burn'], dtype=object)

In [12]:
NSW_gdf.Label.unique()

array(['1968-69 Prescribed Burn', '1970-71 Prescribed Burn',
       '1971-72 Prescribed Burn', '1972-73 Prescribed Burn',
       '1974-75 Prescribed Burn', '1975-76 Prescribed Burn',
       '1976-77 Prescribed Burn', '1977-78 Prescribed Burn',
       '1978-79 Prescribed Burn', '1979-80 Prescribed Burn',
       '1980-81 Prescribed Burn', '1981-82 Prescribed Burn',
       '1982-83 Prescribed Burn', '1983-84 Prescribed Burn',
       '1984-85 Prescribed Burn', '1985-86 Prescribed Burn',
       '1986-87 Prescribed Burn', '1987-88 Prescribed Burn',
       '1988-89 Prescribed Burn', '1989-90 Prescribed Burn',
       '1990-91 Prescribed Burn', '1991-92 Prescribed Burn',
       '1992-93 Prescribed Burn', '1993-94 Prescribed Burn',
       '1994-95 Prescribed Burn', '1995-96 Prescribed Burn',
       '1996-97 Prescribed Burn', '1997-98 Prescribed Burn',
       '1998-99 Prescribed Burn', '1999-00 Prescribed Burn',
       '2000-01 Prescribed Burn', '2001-02 Prescribed Burn',
       '2002-03 Prescrib

In [2]:
#Choose only prescribed burns
SA_gdf = SA_gdf[SA_gdf['INCIDENTTY'].str.contains("Prescribed Burn")]
NSW_gdf = NSW_gdf[NSW_gdf['Label'].str.contains("Prescribed Burn")]

### Get the right columns
We want the state code, the year of the burn, the date of the burn, the area and the geometry. The NSW data needs a bit of format work. We can delete all the columns we don't need. 

In [3]:
#Fix the year format for the NSW data
#The column called 'Label' has both the year and the type of burn in it. Split it into seperate columns
NSW_gdf[['year', 'burn type', 'burn']] = NSW_gdf['Label'].str.split(expand=True)
#delete the last 3 characters from the data in the year column as a string to change '1957-58' into '1957'
NSW_gdf['year'] = NSW_gdf['year'].str[:-3]
#Turn the string back into a number (int64 format)
NSW_gdf.year = NSW_gdf.year.astype('int64')

In [3]:
#Delete the columns that we don't need
SA_gdf = SA_gdf.drop(columns=['INCIDENTNU', 'INCIDENTNA', 'INCIDENTTY', 'FINANCIALY', 'SEASON', 'DATERELIAB', 'SHAPE_Leng'])
VIC_gdf = VIC_gdf.drop(columns=['FIRETYPE', 'STRTDATIT', 'CR_DATE', 'FIRE_NO', 'NAME', 'TREAT_TYPE', 
                                'FIRE_SVRTY', 'FIRE_COVER', 'FIREKEY', 'UPDATEDATE', 'CFA_ID', 'METHOD', 
                                'METHD_CMNT', 'DSE_ID', 'ACCURACY', 'DISTRICTID'])
NSW_gdf = NSW_gdf.drop(columns=['FireName', 'FireNo', 'EndDate', 'PerimeterM', 'Verdate', 'burn type', 'burn', 'Label'])

#Arrange the data by year of burn
SA_gdf = SA_gdf.sort_values(by=['FIREYEAR'])
VIC_gdf = VIC_gdf.sort_values(by=['SEASON'])
NSW_gdf = NSW_gdf.sort_values(by=['year'])

#Add a column that has the state code in it
SA_gdf['STATE'] = 'SA'
VIC_gdf['STATE'] = 'VIC'
NSW_gdf['STATE'] = 'NSW'

#rearrange the column order to be consistant
SA_gdf = SA_gdf[['STATE', 'FIREYEAR', 'FIREDATE', 'SHAPE_Area', 'geometry']]
VIC_gdf = VIC_gdf[['STATE', 'SEASON', 'START_DATE', 'AREA_HA', 'geometry']]
NSW_gdf = NSW_gdf[['STATE', 'year', 'StartDate', 'AreaHa', 'geometry']]

#Select only years greater then or equal to 1988
SA_gdf = SA_gdf[SA_gdf.FIREYEAR >= 1988]
VIC_gdf = VIC_gdf[VIC_gdf.SEASON >= 1988]
NSW_gdf = NSW_gdf[NSW_gdf.year >= 1988]

### Get the area of the burn in Hactares
The South Australian data has the area in decimal degrees, which was calculated directly from the polygon. We need Hactares, and converting decimal degrees to hactares is like super hard mathematically, so we'll just calculate it from the polygon, which means we have to change the projection type of the dataframe to make it calculate m2 instead of decimal degrees. After it's changed, it loses it's 2D Geometry which means we can't plot it as JSON. So to get around this, we'll make a new dataframe to calculate the area from and append that area column back on to the original dataframe that still has the 2D geometry. 

In [4]:
#Make a new data frame that converts the EPSG of the original dataframe. This will change the projection from 2D Geographic into something else. 
SA_gdf1 = SA_gdf.to_crs({'init': 'epsg:3107'}) #this is epsg for GDA94 South Australia
#Make a new column that calculates the area of the polygon (in m2, then /10**4 to get Ha) ??? Is this right?
SA_gdf1['cartesian_area'] = SA_gdf1['geometry'].area/10**4

#Append the cartesian_area column back onto the original dataframe (that still has 2D Geographic projection)
SA_gdf['cartesian_area'] = SA_gdf1['cartesian_area'].to_numpy()
#Rearrange columns
SA_gdf = SA_gdf[['STATE', 'FIREYEAR', 'FIREDATE', 'SHAPE_Area', 'cartesian_area', 'geometry']]


/env/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [5]:
#Converting the data to json
data = json.loads(SA_gdf.to_json())

map = ipyl.Map(center=[-28, 148], zoom=6)

label = ipyw.Label(layout=ipyw.Layout(width='100%'))

for feature in data['features']:
    feature['properties']['style'] = {
        'color': 'grey',
        'weight': 1,
        'fillColor': 'grey',
        'fillOpacity': 0.5
    }
layer = ipyl.GeoJSON(data=data, hover_style={'fillColor': 'red'})

def click_handler(event=None, feature=None, id=None, properties=None):
    label.value = str(properties['cartesian_area']) #you can make the value of a column show up by hovering over the polygon
    
    
layer.on_hover(click_handler)
map.add_layer(layer)


ipyw.VBox([map, label])

# We need to evaluate the completeness of the data
This could be done by adding the total areas together for each year and comparing that to the tabular data we have on area burnt per year. 